In [ ]:
import pandas as pd
from datetime import datetime
import zipfile, requests
import csv

# SETUP OPTIONAL PANDAS OPTIONS
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', None)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199



In [ ]:
# STEP 1, GET THE ORIGINAL DATA AND EXTRACT .XPT FILE

# Original file from icandecide. Check site for details.
adva_file = "https://www.icandecide.org/wp-content/uploads/2022/05/FDA-CBER-2021-5683-0123168%20to%20-0126026_125742_S1_M5_c4591001-A-D-adva.zip"

# Download the file, creating the zip file locally.
with requests.get(adva_file) as r:
    r.raise_for_status()
    with open("c4591001-A-D-adva.zip", 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

# Extract the zipfile, do not rename anything. We now have a .xpt file
extracted = zipfile.ZipFile('c4591001-A-D-adva.zip')
extracted.extractall("")

In [ ]:
# STEP 2, CONVERT THE .XPT FILE TO A PANDAS DATAFRAME

# Read the file using pandas
data = pd.read_sas('FDA-CBER-2021-5683-0123168 to -0126026_125742_S1_M5_c4591001-A-D-adva.xpt',chunksize=50000, encoding='utf-8')

# For performance reasons, we create the pandas dataframe by concatenating data
dfs = []
for chunk in data:
    dfs.append(chunk)

# Final dataframe from the downloaded .xpt file
df_final = pd.concat(dfs)

In [113]:
# STEP 3, CONVERT SAS DATE TO PYTHON DATETIME SO WE CAN QUERY IT LATER
df_final['ADT'] = pd.to_timedelta(df_final['ADT'], unit='D') + pd.Timestamp('1960-1-1')

In [114]:
# QUERY DATA 
# Looking for ALL Placebo patients NEGATIVE at VISIT 1
# We add to that ALL Placebo patients POSITIVE at VISIT 3
placebo_neg_pos = df_final[['USUBJID', 'ACTARM','PARAM', 'VISIT', 'AVALC','ADT']][
    ((df_final.ACTARM == 'Placebo') & (df_final.PARAM == 'N-binding antibody - N-binding Antibody Assay')) &
    ((df_final.VISIT == 'V1_DAY1_VAX1_L') & (df_final.AVALC == 'NEG')) | 
    ((df_final.VISIT == 'V3_MONTH1_POSTVAX2_L') & (df_final.AVALC == 'POS') & (df_final.ADT <= datetime(2020, 11, 15)))
    ]

# This step counts the unique subject IDs, returning those which count to two.
# Subjects that appear twice had both v1_ NEGATIVE and v3_ Positive
placebo_neg_pos_results = placebo_neg_pos[placebo_neg_pos.groupby('USUBJID').USUBJID.transform('count') == 2]

# Repeat, this time for vaccinated
vaccinated_neg_pos = df_final[['USUBJID', 'ACTARM','PARAM', 'VISIT', 'AVALC', 'ADT']][
    ((df_final.ACTARM=='BNT162b2 Phase 2/3 (30 mcg)') & (df_final.PARAM == 'N-binding antibody - N-binding Antibody Assay')) &
    ((df_final.VISIT == 'V1_DAY1_VAX1_L') & (df_final.AVALC == 'NEG')) | 
    ((df_final.VISIT == 'V3_MONTH1_POSTVAX2_L') & (df_final.AVALC == 'POS') & (df_final.ADT <= datetime(2020, 11, 15)))
    ]

vaccinated_neg_pos_results = vaccinated_neg_pos[vaccinated_neg_pos.groupby('USUBJID').USUBJID.transform('count') == 2]

# PRINT TO CSV
placebo_neg_pos_results.to_csv('placebo_neg_pos.csv')
vaccinated_neg_pos_results.to_csv('vaccinated_neg_pos.csv')

# PRINT THE RESULTS
print(f"Placebo: {placebo_neg_pos_results.count()/2} Vaccinated: {vaccinated_neg_pos_results.count()/2}")

Placebo: USUBJID    160.0
ACTARM     160.0
PARAM      160.0
VISIT      160.0
AVALC      160.0
ADT        160.0
dtype: float64 Vaccinated: USUBJID    75.0
ACTARM     75.0
PARAM      75.0
VISIT      75.0
AVALC      75.0
ADT        75.0
dtype: float64


In [ ]:
df_final.to_csv("sas.csv", encoding='utf-8', quoting=csv.QUOTE_MINIMAL)

In [ ]:
# READ THE DATA FROM CSV TO PANDAS
# Download the CSV, add it to project folder. Change filepath below, or if you add file it will just work.
df = pd.read_csv(
    filepath_or_buffer="c4591001-A-D-adva.csv",
    low_memory=False,
)

# CONVERT THE ADT DATE TO DATETIME SO WE CAN FILTER IT LATER
df['ADT'] = pd.to_datetime(df['ADT'])

In [ ]:
# QUERY DATA 
# Looking for ALL Placebo patients NEGATIVE at VISIT 1
# We add to that ALL Placebo patients POSITIVE at VISIT 3
placebo_neg_pos = df[['USUBJID', 'ACTARM','PARAM', 'VISIT', 'AVALC','ADT']][
    ((df['ACTARM'] == 'Placebo') & (df['PARAM'] == 'N-binding antibody - N-binding Antibody Assay')) &
    ((df['VISIT'] == 'V1_DAY1_VAX1_L') & (df['AVALC'] == 'NEG')) | 
    ((df['VISIT'] == 'V3_MONTH1_POSTVAX2_L') & (df['AVALC'] == 'POS') & (df['ADT'] <= datetime(2020, 11, 15)))
    ]

# This step counts the unique subject IDs, returning those which count to two.
# Subjects that appear twice had both v1_ NEGATIVE and v3_ Positive
placebo_neg_pos_results = placebo_neg_pos[placebo_neg_pos.groupby('USUBJID').USUBJID.transform('count') == 2]

# Repeat, this time for vaccinated
vaccinated_neg_pos = df[['USUBJID', 'ACTARM','PARAM', 'VISIT', 'AVALC', 'ADT']][
    ((df['ACTARM']=='BNT162b2 Phase 2/3 (30 mcg)') & (df['PARAM'] == 'N-binding antibody - N-binding Antibody Assay')) &
    ((df['VISIT'] == 'V1_DAY1_VAX1_L') & (df['AVALC'] == 'NEG')) | 
    ((df['VISIT'] == 'V3_MONTH1_POSTVAX2_L') & (df['AVALC'] == 'POS')& (df['ADT'] <= datetime(2020, 11, 15)))
    ]

vaccinated_neg_pos_results = vaccinated_neg_pos[vaccinated_neg_pos.groupby('USUBJID').USUBJID.transform('count') == 2]

# PRINT TO CSV
placebo_neg_pos_results.to_csv('placebo_neg_pos.csv')
vaccinated_neg_pos_results.to_csv('vaccinated_neg_pos.csv')

# PRINT THE RESULTS
print(f"Placebo: {placebo_neg_pos_results.count()/2} Vaccinated: {vaccinated_neg_pos_results.count()/2}")